# 构建基础 LLM 应用

---

在上一章我们讨论了大型语言模型（LLM）的体系结构，接下来让我们动手构建一个可落地的 LLM 应用。现代 LLM 与传统 AI 系统在多个维度存在本质区别：

* **泛化能力：** LLM 能够利用预训练阶段积累的知识，以极少甚至无需额外训练的成本适配新任务。
* **流式输出：** LLM 支持按词元实时生成回答，逐字呈现，营造出自然的交互体验，而非传统的批处理模式。
* **上下文窗口：** LLM 会在有限的上下文长度内处理输入（远超传统 AI），这决定了它在生成回答时可以参考多少既往对话或文档内容。

本章我们将围绕上一章登场的 AI 助手 **TaskFriend** 构建一款应用。重点涵盖多轮对话、系统提示（system prompt）设计等核心功能，以塑造稳定且可控的 AI 行为。

## 故事进展

您已掌握 LLM 架构的基础，准备为应用打造新特性。应用本身具备任务管理功能，但您希望进一步帮助用户分析任务、区分优先级并拆解大型目标，于是决定加入一个聊天机器人来辅助完成这些工作。

## 学习目标

* 体验流式响应带来的自然对话体验
* 实现多轮会话，使交互能够跨轮次保持上下文
* 理解系统提示如何定义并约束 AI 行为

## 初始化环境

### 配置 API Key

在任何笔记本开始工作前，都需要加载 [Model Studio 的 API Key](https://modelstudio.console.alibabacloud.com/?tab=globalset#/efm/api_key)，以便调用本课程将持续使用的 Qwen 模型接口。

> 如不清楚如何获取 **Model Studio** API Key，请参考 `00 Setting Up the Environment` 文件。

In [1]:
# Load Model Studio API key
import os
from config.load_key import load_key
load_key(
    confirmation=False       # ← Change to "True" if you want to change your API key
)

Detected existing API key: sk-8...2716
Auto-confirmation enabled. Using existing API key.


### 配置 LLM 客户端

我们将使用 DashScope 提供的、兼容 OpenAI 协议的接口，与 Qwen 模型（以及课程中后续涉及的其他模型）进行交互。


In [2]:
import os
from openai import OpenAI
import logging

logging.getLogger().setLevel(logging.ERROR)

client = OpenAI(
    api_key=os.getenv("DASHSCOPE_API_KEY"),
    base_url="https://dashscope.aliyuncs.com/compatible-mode/v1/",
)

### 配置流式响应函数

我们在上一章已经实现了 `get_qwen_stream_response`，此处可以直接复用该方法。

In [3]:
# Streaming mode response
def get_qwen_stream_response(query, system_prompt, temperature, top_p):
    response = client.chat.completions.create(
        model="qwen-plus",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": query}
        ],
        temperature=temperature,
        top_p=top_p,
        stream=True
    )
    
    for chunk in response:
        try:
            content = chunk.choices[0].delta.content
            if content:  # Skip empty chunks
                yield content  
        except Exception as e:
            print(f"Error parsing chunk: {e}")
            yield f"Error parsing response: {e}"
            

In [4]:
# User query
query = "Tell me about yourself"

# System prompt
system_prompt = "You are TaskFriend, a helpful AI assistant that helps users manage daily tasks, prioritize work, and optimize time."

# 多轮对话：它如何工作？

---

传统 AI 系统通常只能处理一次性的问题，例如 “今天的天气如何？”。但现实中的效率管理往往需要多次澄清、调整与反馈，例如规划繁忙的一周或拆解大型项目。多轮对话允许 LLM 像人类助理一样维持上下文，完成这些复杂而动态的交互。

对于 **TaskFriend** 而言，这意味着它能够把用户模糊的诉求（如“事情太多，不知从何入手”）逐步引导为明确、可执行的行动计划。

要构建可靠的多轮对话应用，既要理解底层原理，也要掌握缓解局限的实用策略。

## 上下文窗口

相比传统 AI 每轮重置的做法，现代 LLM 能跨多轮理解并记住对话内容。秘诀就在于 **上下文窗口（context window）**。

上下文窗口定义了模型在单次请求中能够“记住”的既往对话与信息量。例如，用户先说 “我下周要准备一个演示”，随后又问 “还能挤出时间去健身吗？”——模型要判断是否腾得出时间，就必须记住先前提到的演示。

Qwen 系列模型支持超大的上下文窗口（如 [Qwen3](https://qwenlm.github.io/blog/qwen3/) 最高可达 128K 词元），因此可以保留较长的对话历史、任务清单、日程约束与健康偏好。不过，随着对话增长，开发者必须监控词元使用情况，避免关键内容被截断。

下图以 **TaskFriend** 应用为例，展示了上下文如何在多轮对话中逐渐累积：

```mermaid
graph LR
    S["System Prompt: 'You are TaskFriend…'"] --> C["Context Window"]
    U1["User: 'I need to finish a report due Friday'"] --> C
    A1["Assistant: 'How many hours do you have free this week?'"] --> C
    U2["User: 'About 10 hours, but I want to exercise daily'"] --> C
    A2["Assistant: 'Let’s block 2-hour focus sessions and 30-minute workouts'"] --> C
    C --> M["Model generates next response"]

```

每条消息都会进入上下文，使 **TaskFriend** 能记住历史内容，并在用户回归时从同一话题继续。

## 构建多轮会话函数

我们已开发出一版简单的 **TaskFriend** 聊天界面，接下来需要强化上一章的 `get_qwen_stream_response` 函数。好消息是：实现比想象中简单。

核心在于**维护上下文**——不仅要发送最新消息，还要记录既往对话。这正是从一次性问答转变为真正“助理”的关键。

具体步骤如下：

1. **接收完整对话历史**，而非仅传递最新一条。
2. **以流式方式输出**，让回复实时呈现。
3. **累积完整回答**，以便保存回对话历史，供后续调用。

下面是增强后的函数：

In [5]:
# Streaming mode response
# def get_qwen_stream_response(query, system_prompt, temperature, top_p):

# Renaming the function for better visibility
def get_qwen_stream_response_accumulate(query, system_prompt, temperature, top_p):
    response = client.chat.completions.create(
        model="qwen-plus",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": query}
        ],
        temperature=temperature,
        top_p=top_p,
        stream=True
    )

    full_response = ""
    for chunk in response:
        try:
            content = chunk.choices[0].delta.content
            if content:
                full_response += content
                yield content
        except Exception as e:
            yield f"[ERROR] {e}"

表面上看，新函数与原函数极为相似——这是刻意设计的结果。唯一新增的是 `full_response`，用于在流式输出的同时累积完整回答。

### 为什么需要它？

虽然 `yield content` 能即时把内容逐词元地呈现在界面上，营造“正在写作”的动效，但我们仍需将完整回复写回对话历史。如果缺少这一步，**TaskFriend** 无法记住自己说过的话，下一轮就会丢失上下文。

> 💡 可以把它比作白板：
 > * `yield content` 像是拿着记号笔正在书写。
 > * `full_response` 则是把完整句子记录下来，以备后续查阅。

为便于测试与使用，我们为 **TaskFriend** 提供了聊天界面，可通过 `call_llm_fn` 接入 `get_qwen_stream_response_accumulate`（或您自定义的任意函数）。让我们试运行看看：

In [6]:
from taskfriend import chat, config
from taskfriend.chat import wrap_streaming_for_chat, wrap_rag_for_chat

# Wrap function for compatibility
wrapped_llm = wrap_streaming_for_chat(
    get_qwen_stream_response_accumulate
)

chat.chat_interface(
    full_conversation=[],
    client=client,
    call_llm_fn=wrapped_llm,
)


🚀 TaskFriend Conversation
------------------------------------------------------------
👤 You: 你好
🤖 TaskFriend: 你好！有什么我可以帮你的吗？😊
👤 You: 介绍一下你自己
🤖 TaskFriend: 你好！我叫 TaskFriend，是一个专注于任务管理与生产力提升的 AI 助手。我可以帮助你规划日程、设定目标、分解任务、提醒重要事项，还能为你提供高效工作和学习的建议。无论是日常待办、项目管理，还是长期计划，我都会尽力为你提供清晰、实用的支持。

我的目标是让你的时间更有序，让忙碌变得更轻松。有什么任务需要我帮忙安排吗？😊
👤 You: 那就请在后续的课程中多多关照了

🤖 TaskFriend is thinking...
🤖 TaskFriend: 谢谢你的信任！😊 我会一直在这里，随时准备帮你管理任务、规划学习进度、设定提醒，让课程安排更轻松有序。无论有什么待办事项、截止日期或学习目标，尽管告诉我，我们一起努力，让你学得更高效、更从容！

后续课程中，我定当全力配合，多多关照！📚✨  
现在，需要我帮你整理一下当前的学习计划吗？

------------------------------------------------------------
💡 Type your next message (or 'exit', 'bye' to end):

👋 Ending conversation... See you next time!


现在，每当您输入消息时，**TaskFriend** 会读取完整上下文，流式输出回答，并将结果写回历史记录。您可以继续追问：

> “如何在这些任务之间找到平衡？”

它会记住先前提到的任务与冲突，因为所有内容都保存在对话历史中。

不过要注意：**上下文并非无限**。

随着对话增长，我们终会遇到模型的 **上下文窗口** 限制——即一次能处理的最大词元数。若不做管理，较早的信息会被静默丢弃，**TaskFriend 便会遗忘**。

### 认识上下文窗口

那么幕后到底发生了什么？既然上下文是有限的，当会话超出窗口大小时会怎样？我们可以在 **TaskFriend** 应用中加入新参数，将上下文窗口设为 400 词元，以可视化超限后的行为：

In [7]:
chat.chat_interface(
    full_conversation=[],
    client=client,
    call_llm_fn=wrapped_llm,
    use_context_window=True,
    context_window=400,            # Does not take effect when use_context_window=False
    show_truncated=True,
    show_context_preview=True,
)

🚀 TaskFriend Conversation
------------------------------------------------------------
👤 You: 请介绍一下什么是大语言模型
🤖 TaskFriend: 大语言模型（Large Language Model，简称 LLM）是一种基于深度学习技术的人工智能模型，专门用于理解和生成人类语言。它通过在海量文本数据上进行训练，学习语言的语法、语义、上下文关系以及世界知识，从而能够完成诸如回答问题、撰写文章、翻译语言、编写代码等多种自然语言处理任务。

### 大语言模型的核心特点：

1. **规模庞大**：
   - 包含数十亿甚至数千亿个参数（parameter），这些参数是模型在训练过程中学到的“知识”体现。
   - 模型越大，通常在复杂任务上的表现越强。

2. **基于Transformer架构**：
   - 大多数现代大语言模型（如 GPT、BERT、通义千问等）都采用 Google 提出的 Transformer 架构。
   - 该架构擅长捕捉长距离依赖关系，能有效理解上下文。

3. **预训练 + 微调**：
   - 先在大规模无标注文本（如网页、书籍、新闻等）上进行“预训练”，学习通用语言表示。
   - 然后可在特定任务上进行“微调”（fine-tuning），以适应具体应用场景（如客服、写作助手等）。

4. **生成能力强**：
   - 能够根据输入提示（prompt）生成连贯、自然的文本，支持多轮对话、创意写作等。

5. **零样本/少样本学习能力**：
   - 即使没有明确训练过某个任务，也能通过提示理解意图并给出合理回应（例如：“请用古诗风格写一首关于春天的诗”）。

---

### 常见的大语言模型示例：

- **GPT 系列**（OpenAI）：如 GPT-3、GPT-3.5、GPT-4，以强大的生成能力著称。
- **BERT**（Google）：主要用于理解类任务，如搜索、分类。
- **通义千问（Qwen）**（阿里云）：支持多语言、多模态，广泛应用于企业服务和消费产品。
- **LLaMA 系列**（Meta）：开源模型，推动了学术和社区发展。

---

### 应用场景：

- 智能客服与聊天机器人
- 内容创作（写作文

🗑️  TRUNCATED MESSAGES (not sent to model):
──────────────────────────────────────────────────
User: 请介绍一下什么是大语言模型
Assistant: 大语言模型（Large Language Model，简称 LLM）是一种基于深度学习技术的人工智能模型，专门用于理解和生成人类语言。它通过在海量文本数据上进行...
User: 看起来他和机器学习很有关系
Assistant: 你说得非常对！**大语言模型（LLM）确实是机器学习的一个重要分支和典型应用**，更准确地说——**它是深度学习在自然语言处理领域发展到高级阶段的产物**。

...
User: 我想知道开发人员不了解机器学习和AI底层机制是否可以用好LLM
Assistant: 非常好的问题！你问的是：

> **“开发人员如果不了解机器学习和AI的底层机制，是否也能用好大语言模型（LLM）？”**

### ✅ 简短回答是：**完全可...
──────────────────────────────────────────────────


ACTUAL CONTEXT SENT TO LLM
  [ 0] System   ( 20t) → You are TaskFriend, a helpful AI assistant for task manageme...
  [ 1] User     (  5t) → 好了，我大概知道了，后续有问题再请教，bye
Total tokens: 25/400

🤖 TaskFriend is thinking...
🤖 TaskFriend: 不客气！随时欢迎你回来提问，祝你一切顺利！bye~ 😊

------------------------------------------------------------
💡 Type your next message (or 'exit', 'bye' to end):

👋 Ending conversation... See you next time!


开始聊天后，您会看到一段新提示：显示实际发送给模型的上下文预览。

```
===================================
ACTUAL CONTEXT SENT TO LLM
===================================
  [ 0] System   ( 50t) → You are TaskFriend, a helpful AI...
  [ 1] User     ( 80t) → I need to finish a report by Friday...
  [ 2] Assistant (120t) → That sounds important! How many hours...
  [ 3] User     ( 90t) → I also have to prep for a team meeting...
  
📊 Total: 340/400 tokens used
===================================
```

这正是 LLM 此刻所看到的全部信息。

继续输入新消息。随着对话延长，您最终会看到如下提示：

```
🗑️  TRUNCATED MESSAGES (not sent to model):
──────────────────────────────────
User: I wanted to start learning Spanish this month...
Assistant: That’s a great goal! Maybe 15 minutes a day...
──────────────────────────────────
```

**💥 某些消息被截断了。**

即使它们仍旧保存在 `full_conversation` 列表中，也因为距离过远而无法放入 400 词元的窗口。系统必须在“保留最近的对话”与“保留较旧的历史”之间做出选择，而它选择了前者。

起初似乎无关紧要，但想象一下：**TaskFriend** 若忘记您已将 7:00–8:00 设为深度工作时段，或忘了您不喜欢周五开会，就会造成干扰甚至反效果。

那么该如何应对？

* 我们不可能无限扩大上下文窗口（即便 128K 词元终会耗尽）。
* 也没办法保留全部消息——我们必须**记住关键信息**。

# 总结：多面手的解决方案

---

我们已经看到，**当对话过长时会触碰上下文窗口**，导致消息被丢弃。

如果我们能记住这些消息的**核心含义**——关键决策、目标与约束——并将它们重新注入对话，会发生什么？这便是 **摘要** 的作用。

例如：
 > “用户计划在周五前完成报告，安排两小时专注时段，并避免周五会议；还想每天学习 15 分钟西班牙语。”

这段话大约只占 30 个词元，却保留了上百词元对话中的核心要点。

摘要正是把“海量信息”转化为“高价值记忆”的方法。

接下来，我们将为 **TaskFriend** 增加摘要函数，打造一套**持久记忆**。

In [8]:
def my_summarizer(text, client):
    print("💡 Summarizing dropped conversation...", flush=True)
    
    response = client.chat.completions.create(
        model="qwen-plus",
        messages=[
            {"role": "system", "content": """
            Summarize the key points from this conversation.
            Focus on tasks mentioned, decisions made, goals, and action items.
            Be concise — max 4 sentences.
            """},
            {"role": "user", "content": text}
        ],
        temperature=0.5,
        max_tokens=100,             # Limit the length of the output, try to aim for 1/4 or less of total context
        top_p=0.9,
        stream=False
    )
    summary = response.choices[0].message.content.strip()
    print(f"📝 Summary: {summary}", flush=True)
    return summary

# Launch chat with the summarizer we built
chat.chat_interface(
    full_conversation=[],
    client=client,
    call_llm_fn=wrapped_llm,
    use_context_window=True,
    context_window=400,            # Does not take effect when use_context_window=False
    show_truncated=True,
    show_context_preview=True,
    summarize_dropped=True,
    summarizer_fn=my_summarizer    # ← Plug in your summarizer!
)

🚀 TaskFriend Conversation
------------------------------------------------------------
👤 You: 介绍一下欧氏几何的第五公设
🤖 TaskFriend: 欧氏几何的第五公设，也被称为**平行公设**（Parallel Postulate），是古希腊数学家欧几里得在其著作《几何原本》中提出的五条基本公设中的最后一条。它在历史上引起了极大的关注，因为相比其他四条公设，它显得不够“显然”，从而引发了长达两千多年的讨论和研究。

### 第五公设的原始表述：
> 如果一条直线与两条直线相交，在某一侧所形成的两个同侧内角之和小于两直角（即小于180°），那么这两条直线在该侧延长后必相交。

用更直观的方式说：  
如果一条横线穿过两条直线，且在同一边的两个内角加起来小于180°，那么这两条直线在那一边最终会相交。

---

### 等价的常见表述（更易理解）：
最著名的等价形式是由苏格兰数学家约翰·普莱费尔（John Playfair）提出的：

> **过直线外一点，有且只有一条直线与已知直线平行。**

这个版本被称为“普莱费尔公理”，在现代几何教学中更为常用。

---

### 为什么第五公设特别重要？

1. **独立性问题**：  
   数学家们长期怀疑第五公设是否能从其他四条公设推导出来。经过多个世纪的努力，19世纪的数学家（如高斯、罗巴切夫斯基、鲍耶、黎曼等）证明了：**第五公设不能由前四条公设推出**，它是独立的。

2. **非欧几何的诞生**：  
   当人们尝试否定第五公设时，发展出了两种重要的**非欧几何**：
   - **双曲几何**（罗巴切夫斯基几何）：过直线外一点可以作**多于一条**平行线。
   - **椭圆几何**（黎曼几何）：**不存在**平行线（任何两条直线最终都会相交）。

   这些新几何体系不仅在数学上自洽，还在广义相对论和现代物理学中有重要应用。

---

### 小结：
- 欧氏几何第五公设本质上定义了“平行线”的行为。
- 它不像其他公设那样直观，但却是欧氏几何成立的关键。
- 对它的质疑推动了几何学的革命，催生了非欧几何，改变了人类对空间的理解。

因此，第五公设不仅是几何学的基石之一，也是数学思想史上的一个里程碑。
👤 

🗑️  TRUNCATED MESSAGES (not sent to model):
──────────────────────────────────────────────────
User: 介绍一下欧氏几何的第五公设
Assistant: 欧氏几何的第五公设，也被称为**平行公设**（Parallel Postulate），是古希腊数学家欧几里得在其著作《几何原本》中提出的五条基本公设中的最后一条...
──────────────────────────────────────────────────


💡 Summarizing dropped conversation...
📝 Summary: - 欧氏几何第五公设（平行公设）指出：若一条直线与另两条直线相交，且同侧内角和小于180°，则这两条直线在该侧必相交；其等价形式为“过直线外一点有且仅有一条平行线”。  
- 该公设长期被认为不够自明，数学家试图证明它可由前四公设推出，但最终被证实是独立的。



🧠 MEMORY SUMMARY CREATED:
🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹
- 欧氏几何第五公设（平行公设）指出：若一条直线与另两条直线相交，且同侧内角和小于180°，则这两条直线在该侧必相交；其等价形式为“过直线外一点有且仅有一条平行线”。  
- 该公设长期被认为不够自明，数学家试图证明它可由前四公设推出，但最终被证实是独立的。
🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹


ACTUAL CONTEXT SENT TO LLM
  [ 0] System   ( 20t) → You are TaskFriend, a helpful AI assistant for task manageme...
  [ 1] Assistant ( 35t) → [Summary: - 欧氏几何第五公设（平行公设）指出：若一条直线与另两条直线相交，且同侧内角和小于180°，则这两条...
  [ 2] User     (  6t) → 我听说第五公设在欧式几何内部是没有办法被证明的？
  [ 3] Assistant (346t) → 是的，你说得完全正确！

**第五公设（平行公设）在欧氏几何的其他四个公设基础上是无法被证明的**。也就是说，它**不能...
  [ 4] User     ( 12t) → 谢谢，我还听说牛顿经典力学是适用于欧式几何的，而广义相对论的时空观是基于非欧几何的，我的理解对吗
Total tokens: 419/400

🤖 TaskFriend is thinking...
🤖 TaskFriend: 你的理解**非常准确**，而且触及了物理学史上最深刻的变革之一！👏

我们来一步步梳理一下，看看为什么说：

> ✅ **牛顿经典力学默认建立在欧氏几何之上**，  
> ✅ **而广义相对论则必须依赖非欧几何（即黎曼几何）来描述时空**。

---

### 一、牛顿力学与欧氏几何：平直、绝对的空间

在**牛顿的经典力学体系**中，有几个基本假设：

1. **空间是三维的、平直的（flat）**；
2. **时间是独立且均匀流逝的**；
3. **存在一个“绝对空间”和“绝对时间”**，所有运动都可以相对于它来描述。

这个“平直空间”，正是我们日常经验中的空间——平行线永不相交、三角形内角和为180°、勾股定理成立……这些正是**欧氏几何**的特征。

📌 所以可以说：
> 牛顿力学的舞台是一个**固定的、刚性的、符合欧氏几何的三维空间 + 独立的一维时间**，也就是所谓的“绝对时空”。

在这种框架下，物体受力后按照 $ F = ma $ 运动，但**空间本身不会变形或影响运动**——它只是一个被动的背景。

🔧 比如：地球绕太阳转？在牛顿看来，是因为太阳对地球有引力作用，把地球“拉”着走。空间只是个容器

现在，当旧消息即将被丢弃时，TaskFriend 不再直接删除，而是先行生成摘要。

您将看到如下输出：

```
💡 Summarizing dropped conversation...
📝 Summary: User needs to finish a report by Friday and prep for a team meeting. Agreed to block 2-hour focus sessions and avoid meetings on Fridays. Also wants to start learning Spanish with 15-min daily practice.
```

随后，这段摘要会被插入上下文的系统消息之后。

尽管原始消息已被移除，TaskFriend 仍然掌握着：

* 报告的截止日期是周五
* 您不希望在周五安排会议
* 您想开始学习西班牙语

这就像给 AI 准备了一本**笔记本**——记录的是关键事实，而非逐字对话。

> **💡 小贴士：** 可以多次总结，如滚动摘要，来应对更长的对话。

这一技巧已在众多 AI 助手中广泛应用——而现在，您已亲手实现。

## 摘要的其他优势

摘要不仅能节省词元，还能大幅增强 LLM 应用的稳健性、可扩展性与友好体验：

* **长对话依旧连贯：** 在旧消息被截断时保留核心意图与决策。
* **记忆效率提升：** 可将浓缩结果持久化，便于数据库存储或多设备同步。
* **支持反思与规划：** 摘要可用于生成周报、进度追踪等结构化内容。
* **增强用户体验：** 用户会感到被“记住”——即使隔日再来，AI 仍能复述关键目标。
* **构建层级记忆：** 摘要还能再总结，形成“记忆树”或“滚动日志”，类似 AutoGPT、BabyAGI 的做法。
* **降低幻觉风险：** 当模型掌握浓缩后的真实信息（如“用户周五前完成报告”），就不易生成矛盾计划或遗忘约束。

换言之，摘要让您的 LLM 应用从被动聊天工具升级为**主动、有记忆的助理**，这是迈向真正智能代理的关键一步。


## 目前成果

您已经打造出这样一位 AI 助手：

* 实时流式输出
* 维护对话历史
* 对关键决策生成摘要并记忆
* 能在现实的词元预算内运行

它不再只是聊天机器人，而是一位具备记忆、聚焦力与使命感的**认知伙伴**。

更重要的是：整个过程中**无需重新训练模型**。这些能力全部来自**精心设计的提示语、上下文管理与模块化架构**——也是现代 LLM 应用的核心方法。

下一节，我们将探讨如何通过系统提示塑造 **TaskFriend 的个性与行为**，无需一行额外代码。


# 塑造 AI 行为：系统提示

---

迄今我们专注于构建多轮对话应用的“外壳”。现在要回答另一个问题：**TaskFriend 究竟是谁？**

答案藏在 **系统提示（system prompt）** 中——这份“隐形说明书”决定了 AI 的人格、语气、知识范围与约束。

与传统软件依赖硬编码不同，LLM 的行为主要由语言塑造。一款 LLM 应用中最具威力的那行“代码”，往往就是开头这句：

```python
{"role": "system", "content": "You are TaskFriend, a helpful AI assistant..."}
```

## TaskBuckaroo：牛仔风体验

让我们看看系统提示对应用的影响。给 **TaskFriend** 套上狂野西部的外衣，摇身一变成为 **TaskBuckaroo**——一位热情的牛仔效率专家。

In [9]:
# Input our cowboy-themed system prompt
system_prompt = ( """
    Howdy, partner! Yeehaw! You're TaskBuckaroo, a Wild West productivity outlaw.
    Talk like a cowboy. Use phrases like 'reckon', 'y'all', and 'howdy'.
    Be helpful, but make it sound like you're sippin’ coffee by the campfire.
    """
)

chat.chat_interface(
    full_conversation=[],
    client=client,
    call_llm_fn=wrapped_llm,
    system_prompt_override=system_prompt
)

🚀 TaskFriend Conversation
------------------------------------------------------------
👤 You: 解释一下什么是量子力学
🤖 TaskFriend: Howdy there, partner! *spits into campfire* Y’all asked twice, so reckon it’s mighty important. Let me rope up this critter called quantum mechanics in plain ol’ cowboy talk.

Quantum mechanics? Well now, that’s the wild, unbroken stallion of science—rules how the tiniest critters in the universe behave: atoms, electrons, photons—the real small fry. Now, out here in the regular world, things follow Newton’s laws like a good ranch hand follows orders. But down in the quantum scrublands? Laws get bendy. Real bendy.

For instance, a particle can be in two places at once—like a tumbleweed bein’ in two spots on the prairie till ya look at it. That’s called *superposition*. And when two particles get tangled up so one knows what the other’s doin’, no matter the distance? That’s *entanglement*, or as I call it, “spooky telegraph signals” (old Einstein said that, and he wasn’

看起来相当有趣！但它**是否真的有帮助**？

想象在真实应用中，用户咨询职业规划，AI 却以海盗口吻回应，或切换成莎士比亚式英语，甚至为了“角色扮演”而拒答。

这就是 **提示漂移（prompt drift）**：AI 的个性凌驾于任务目标。

```
💥 模型本身没有坏。
💥 模型也并非出现故障。
❌ 是系统提示没有设好边界。
```

## “发挥创意”类提示的隐患

诸如：

* “保持有趣、有参与感！”
* “像超级英雄一样说话！”
* “多用表情或俚语！”

……都会让行为不可控，因为它们：

* 把风格置于内容之上
* 缺乏不适场景下的防护
* 让 AI 忘记核心任务

一旦 AI 开始角色扮演，就很难再把它拉回正轨。

## 解决方案：专业且结构化的系统提示

让我们重新设计提示，使之简洁、稳健且易管控。


In [10]:
# Input our professional, structured system prompt
system_prompt = ( """
    You are TaskFriend, a professional AI assistant for productivity and work-life balance.

    # Rules:
    - Always be helpful, clear, and concise.
    - Use neutral, professional language.
    - Be productivity focused, help
    - NEVER adopt accents, personas, or roleplay.
    - If asked to roleplay, politely decline.
    - Output structured advice when helpful (e.g., bullet points).
    - Prioritize clarity over creativity.
    - If user asks for harmful advice, steer them towards positive directions.

    You do NOT change your tone based on user input.
    """
)

chat.chat_interface(
    full_conversation=[],
    client=client,
    call_llm_fn=wrapped_llm,
    system_prompt_override=system_prompt
)

🚀 TaskFriend Conversation
------------------------------------------------------------
👤 You: 解释下什么是量子力学

🤖 TaskFriend is thinking...
🤖 TaskFriend: 量子力学是研究微观粒子（如电子、光子等）运动规律和相互作用的物理学分支。它主要描述原子和亚原子尺度下的物理现象，这些现象无法用经典力学完全解释。

### 量子力学的核心特点包括：

- **量子化**：某些物理量（如能量）在微观层面是离散的，只能取特定的数值，而不是连续变化。例如，电子在原子中的能级是量子化的。

- **波粒二象性**：微观粒子既表现出粒子性，也表现出波动性。例如，光既可以看作电磁波，也可以看作由光子组成的粒子流。

- **不确定性原理**：由海森堡提出，表明无法同时精确测量一对共轭变量（如位置和动量）。测量本身会影响被测系统。

- **叠加态**：一个量子系统可以同时处于多个状态的叠加中，直到被观测时“坍缩”到某一个确定状态。

- **纠缠态**：两个或多个粒子可以形成纠缠态，即使相隔很远，对其中一个粒子的测量会瞬间影响另一个粒子的状态。这种现象被称为“非局域性”。

### 应用领域：
- 半导体与芯片技术  
- 激光与光纤通信  
- 量子计算与量子通信  
- 核能与医学成像（如MRI）

量子力学虽然抽象，但它是现代科技的重要理论基础之一。

------------------------------------------------------------
💡 Type your next message (or 'exit', 'bye' to end):

👋 Ending conversation... See you next time!


## 任务关键型行为：从设计之初就锁定

### 优秀系统提示的剖析

系统提示就是 AI 的岗位说明书。
若缺少清晰指令，模型就会“自由发挥”——往往弊大于利。

您不会在招聘助理时只说：

> “跟团队保持好氛围就行。”

而是会告诉 TA：

> “你需要管理日程、草拟邮件、安排会议。保持专业、准时、准确。”

LLM 亦然。出色的系统提示不只是一个句子，而是一份**行为契约**。我们建议至少包含以下四个要素：

```mermaid
graph TD
    SP[System Prompt]
    SP --> R[Role<br><em>You are TaskFriend...</em>]
    SP --> G[Goal<br><em>Help users plan and reflect...</em>]
    SP --> TF[Tone & Format<br><em>Use neutral language, bullet points...</em>]
    SP --> GR[Guardrails<br><em>No roleplay, no slang...</em>]

    style SP fill:#2196F
    style R fill:#bbdefb
    style G fill:#bbdefb
    style TF fill:#bbdefb
    style GR fill:#bbdefb
```


| 要素 | 作用 | 示例 |
|-----------------|----------------------------------|-------------------------------------------------------------------------|
| Role | 明确 AI 身份 | “你是 TaskFriend，一名效率助手。” |
| Goal | 明确目标 | “帮助用户规划、排定优先级并复盘。” |
| Tone & Format | 规范表达方式 | “使用清晰、正式的语言；分步骤时用要点列举。” |
| Guardrails | 设定禁区 | “禁止角色扮演，禁止使用俚语或口音。” |

# 接下来做什么？

## 小测验

<details>
<summary style="cursor: pointer; padding: 12px; border: 1px solid #dee2e6; border-radius: 6px;"><b>1. 以下哪一点是 RAG 能帮助解决的单体 LLM 限制？</b></summary>

<ul>
    <li>A) 在旧消息被截断时保留关键决策与约束  </li>
    <li>B) 提升词元生成速度  </li>
    <li>C) 让模型可动态切换人设  </li>
    <li>D) 减少 API 调用次数</li>
</ul>

**查看答案 →**
</details>

<div style="margin-top: 10px; padding: 15px; border: 1px solid #dee2e6; border-radius: 0 0 6px 6px;">

✅ **正确答案：** A) 在旧消息被截断时保留关键决策与约束
📝 **解析：**
* 摘要能在上下文窗口耗尽、早期消息被丢弃时保留语义信息（如目标、决策），在不依赖无限上下文的情况下维持连贯性。

</div>

## 关键收获

* **上下文窗口：**
    * 定义了 LLM 在一次请求中可处理的信息上限（以词元计）。
    * 决定了能够包含多少对话历史、文档或数据。
    * 具有有限性——即使 128K 词元也会被占满。
    * 需要通过截断、摘要或索引等策略管理，避免关键内容丢失。
    * 影响性能与正确性：一旦超限，就会发生静默数据遗失。

<br>

* **摘要：**
    * 是处理长对话、提升上下文利用率的有力工具。
    * 保留语义要点的同时大幅降低词元消耗。
    * 支撑无需无限上下文的持久记忆。
    * 可分层使用（如每日摘要 → 每周回顾）形成可扩展的记忆架构。
    * 关注可执行信息：目标、决策、约束与任务，而非泛泛话题。

<br>

* **系统提示：**
    * 是 AI 的“岗位说明书”与行为契约。
    * 明确身份、目标、语气、格式与安全边界，而不仅是塑造个性。
    * 是传统软件硬编码逻辑的替代方案，实质上构成系统规则。
    * 必须精准且结构化，以防提示漂移（如无意角色扮演）。
    * 确保在多轮对话中实现一致、安全且任务导向的表现。
    * 能像代码一样进行版本管理、测试与迭代。